In [2]:
import sqlite3
import pandas as pd
import os

In [4]:
db = os.path.abspath('database.db')

In [6]:
#check the databse tables
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    cursor.execute("select name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:", tables)
    for table in tables:
        query = f'select * from {table[0]}' 
        df = pd.read_sql_query(query, conn)
        print(df.head())

Tables in the database: []


In [203]:
# # Drop all tables 
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    cursor.execute("select name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table in tables:
        cursor.execute(f"drop table if exists {table[0]}")
    conn.commit()

In [204]:
#read: https://www.sqlite.org/lang_datefunc.html
with sqlite3.connect(os.path.abspath(db)) as conn:
    cursor = conn.cursor()
    #text 
    cursor.execute('create table if not exists table1(d1 text)')
    cursor.execute("insert into table1(d1) values (datetime('now'))")
    #real 
    cursor.execute("create table if not exists table2(d1 real)")
    cursor.execute("insert into table2(d1) values (julianday('now'))")

    #int 
    cursor.execute('create table if not exists table3(d1 int)')
    cursor.execute("insert into table3(d1) values (strftime('%s','now'))")

    query1 = 'select *, typeof(d1) from table1'
    query2 = 'select *, typeof(d1) from table2'
    query3 = "select *, typeof(d1) from table3"
    #instead of df, use: cursor.execute(query).fetchall()
    df1 = pd.read_sql_query(query1, conn)
    df2 = pd.read_sql_query(query2, conn)
    df3 = pd.read_sql_query(query3, conn)

    

In [205]:
with sqlite3.connect(db) as conn:
    cursor = conn.cursor()
    cursor.execute("alter table table1 add 'title' varchar")
    cursor.execute("alter table table1 add 'rating' varchar")
    cursor.execute("alter table table1 add 'cost' int")
    cursor.execute("insert into table1('title', 'rating','cost') values('NUTS', 'G', 29), ('Park','G',30),('Extra', 'G', 22),('Control', 'PG', 20),('excited', 'G', 39),('Control', 'PG', 20),('excited', 'G', 19),('City', 'PG', 10),('excited', 'G', 19)")
    table1 = cursor.execute("select *, typeof(cost) from table1").fetchall()
table1



[('2024-10-22 00:15:49', None, None, None, 'null'),
 (None, 'NUTS', 'G', 29, 'integer'),
 (None, 'Park', 'G', 30, 'integer'),
 (None, 'Extra', 'G', 22, 'integer'),
 (None, 'Control', 'PG', 20, 'integer'),
 (None, 'excited', 'G', 39, 'integer'),
 (None, 'Control', 'PG', 20, 'integer'),
 (None, 'excited', 'G', 19, 'integer'),
 (None, 'City', 'PG', 10, 'integer'),
 (None, 'excited', 'G', 19, 'integer')]

In [ ]:
#windows function
#RANK vs ROW_NUMBER: RANK assigns the same unique values for ties and assigns the next value with the total rows up to that row. 
with sqlite3.connect(db) as conn:
    cursor = conn.cursor()
    #test index cost for optimization
    cursor.execute("create index cost_idx on table1('cost')")
    table = cursor.execute("select *, avg(cost) over(partition by rating) as avg_cost from table1").fetchall()
    print(table)
    table = cursor.execute("select *, max(cost) over(partition by rating) as max_cost, sum(cost) over() as overall from table1").fetchall()
    print(table)
    table = cursor.execute("select *, row_number() over(order by cost desc) as row_cost from table1").fetchall()
    print(table)
    table = cursor.execute("select *, row_number() over(partition by rating) as row_rating_cost from table1").fetchall()
    print(table)
    table = cursor.execute("select *, rank() over(order by cost desc) as rank_cost from table1").fetchall()
    print(table)
    table = cursor.execute("select *, rank() over(partition by rating order by cost desc) as rank_rating_cost from table1").fetchall()
    print(table)
    table = cursor.execute("select *, dense_rank() over(order by cost desc) as denserank_rating_cost from table1").fetchall()
    print(table)